In [8]:
import os

import numpy as np
from tensorflow.keras.models import *

from src.dlla.berg import make_mlp
from src.dlla.hw import prepare_traces_dl, dlla_known_p
from src.dlla.wegener import make_mlp_wegener, wegener_p
from src.tools.la import shuffle
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW

In [9]:
from src.tvla.cri import rho_test, tvla_t_test


def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [10]:
LIMITS = {
    Database.ascad: (None, 1000),
    Database.ascad_none: (None, 1000),
    Database.aisy: (1000, 1000),
}

PARAMS = {
    Database.ascad: {
        PollutionType.desync: np.arange(0, 2.05, .05),
        PollutionType.gauss: np.arange(0, 5.1, .1)
    },
    Database.ascad_none: {
        PollutionType.desync: np.arange(0, 205, 5),
        PollutionType.gauss: np.arange(0, 102, 2)
    },
    Database.aisy: {
        PollutionType.desync: np.arange(0, 460, 10),
        PollutionType.gauss: np.arange(0, 4100, 100)
    }
}

def get_params(db: Database, pollution_type: PollutionType):
    return PARAMS[db][pollution_type]

def get_limit(db: Database):
    return LIMITS[db]

def get_order(db: Database):
    if db is Database.ascad:
        return 2

    return 3

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        prof_lim, att_lim = get_limit(db)
        trace_set = TraceSetHW(db, pollution, (prof_lim, att_lim))

        if os.path.exists(trace_set.path):
            # print("Prepare traces (1/3)", end="\r")
            # x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            # print("Prepare traces (3/3)", end="\r")
            # x2b, y2b, x2b_att, y2b_att = prepare_traces_dl(*trace_set.profile_la(), *trace_set.attack_la(True), num_classes=2)
            #
            # print("Make model (1/3)    ", end="\r")
            # mdl_open = make_mlp(x9, y9, progress=False)
            # print("Make model (3/3)    ", end="\r")
            # mdl_wegener = make_mlp_wegener(x2b, y2b, False)

            tvla_x, tvla_y = shuffle(*trace_set.profile_la())
            print("TVLA                ", end="\r")
            # tvla_ps = np.min(tvla_t_test(tvla_x[:att_lim], tvla_y[:att_lim], get_order(db), progress=False), axis=1)
            rho_p = np.min(rho_test(tvla_x[:att_lim], tvla_y[:att_lim], progress=False), axis=1)[0]

            # print("Predict             ", end="\r")
            # dlla_k_p = dlla_known_p(mdl_open, x9_att, y9_att)
            # dlla_wegener_p = wegener_p(mdl_wegener, x2b_att, y2b_att)

            title = f"{pollution_type} ({param})."
            print(f"{title} TVLA rho ({rho_p}).")
            # print(f"{title} TVLA t ({tvla_ps}). TVLA rho ({rho_p}). DLLA known key ({dlla_k_p}). DLLA Wegener ({dlla_wegener_p})")

            # for order, p in enumerate(tvla_ps):
            #     if order > 0:
            #         store_results(db, f"tvla_t_{order}", pollution, p)

            store_results(db, "tvla_rho", pollution, rho_p)
            # store_results(db, "dlla_open", pollution, dlla_k_p)
            # store_results(db, "dlla_wegener", pollution, dlla_wegener_p)

        # print()

la_benchmark(Database.ascad_none, PollutionType.desync, [0])

PollutionType.desync (0). TVLA rho (5.736683290610134e-134).


In [ ]:
def la_benchmark_all(db: Database):
    for poll in [PollutionType.desync, PollutionType.gauss]:
        la_benchmark(db, poll, get_params(db, poll))

while True:
    la_benchmark_all(Database.aisy)
    la_benchmark_all(Database.ascad)
    la_benchmark_all(Database.ascad_none)

PollutionType.desync (0). TVLA rho (8.504552810546213e-12).
PollutionType.desync (10). TVLA rho (7.838330881175729e-11).
PollutionType.desync (20). TVLA rho (1.0356954702000692e-11).
PollutionType.desync (30). TVLA rho (4.39427337024717e-11).
PollutionType.desync (40). TVLA rho (5.431272827399694e-10).
PollutionType.desync (50). TVLA rho (2.3946726204923293e-09).
PollutionType.desync (60). TVLA rho (5.361511092970994e-09).
PollutionType.desync (70). TVLA rho (1.7829890790463252e-10).
PollutionType.desync (80). TVLA rho (6.015952887426258e-10).
PollutionType.desync (90). TVLA rho (6.165592455581402e-10).
PollutionType.desync (100). TVLA rho (1.5861543079181358e-08).
PollutionType.desync (110). TVLA rho (1.3933942108271882e-07).
PollutionType.desync (120). TVLA rho (3.4799560062900175e-09).
PollutionType.desync (130). TVLA rho (1.2608384094065531e-06).
PollutionType.desync (140). TVLA rho (3.441671144352356e-08).
PollutionType.desync (150). TVLA rho (9.813917862171963e-08).
PollutionType